In [1]:
import pandas as pd
import pickle
import re
import nltk
# Load datasets
links = pd.read_csv("../data/links.csv")
movies = pd.read_csv("../data/movies.csv")
ratings = pd.read_csv("../data/ratings.csv") 
tags = pd.read_csv("../data/tags.csv")
# genome_tags = pd.read_csv('/content/drive/MyDrive/ml-20m-mix/genome-tags.csv')
# genome_scores = pd.read_csv('/content/drive/MyDrive/ml-20m-mix/genome-scores.csv')

# Display the first few rows of each DataFrame to understand the structure
print("Ratings:")
print(ratings.head())

print("\nTags:")
print(tags.head())

print("\nMovies:")
print(movies.head())

print("\nLinks:")
print(links.head())

# print("\nGenome Tags:")
# print(genome_tags.head())

# print("\nGenome Scores:")
# print(genome_scores.head())


Ratings:
   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580

Tags:
   userId  movieId            tag   timestamp
0      18     4141    Mark Waters  1240597180
1      65      208      dark hero  1368150078
2      65      353      dark hero  1368150079
3      65      521  noir thriller  1368149983
4      65      592      dark hero  1368150078

Movies:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                

In [3]:
display(ratings.info())
display(tags.info())
display(movies.info())
display(links.info())
# display(genome_tags.info())
# display(genome_scores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   userId     465564 non-null  int64 
 1   movieId    465564 non-null  int64 
 2   tag        465548 non-null  object
 3   timestamp  465564 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 14.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


None

In [2]:
# Handling missing values
tags.dropna(subset=['tag'], inplace=True)
links.dropna(subset=['tmdbId'], inplace=True)

# Converting timestamps to datetime (if needed for analysis)
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
tags['timestamp'] = pd.to_datetime(tags['timestamp'], unit='s')

# Removing duplicates if any
ratings.drop_duplicates(inplace=True)
tags.drop_duplicates(inplace=True)
movies.drop_duplicates(inplace=True)
links.drop_duplicates(inplace=True)
# genome_tags.drop_duplicates(inplace=True)
# genome_scores.drop_duplicates(inplace=True)

# Display cleaned data info
display(ratings.info())
display(tags.info())
display(links.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userId     int64         
 1   movieId    int64         
 2   rating     float64       
 3   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 610.4 MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 465548 entries, 0 to 465563
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userId     465548 non-null  int64         
 1   movieId    465548 non-null  int64         
 2   tag        465548 non-null  object        
 3   timestamp  465548 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 17.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 27026 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27026 non-null  int64  
 1   imdbId   27026 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 844.6 KB


None

In [3]:
from surprise import Dataset, Reader

# Define the reader format for Surprise
reader = Reader(rating_scale=(0.5, 5.0))

# Load the ratings dataset into Surprise's Dataset format
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Splitting the data into training and testing (e.g., 80% training, 20% testing)
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

print("Data has been prepared for collaborative filtering.")


Data has been prepared for collaborative filtering.


In [5]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate

# Instantiate the SVD model
svd = SVD()

# Train the model on the training set
svd.fit(trainset)

# Make predictions on the test set
predictions = svd.test(testset)

# Calculate RMSE to evaluate model performance
rmse = accuracy.rmse(predictions)
print(f"RMSE of the SVD model: {rmse:.4f}")


RMSE: 0.7865
RMSE of the SVD model: 0.7865


In [1]:
# Save Model
filename = "../models/20M_Model_SVD.sav"
pickle.dump(svd, open(filename, "wb"))

NameError: name 'pickle' is not defined

In [6]:
# Test loading
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
filename = "../models/20M_Model_SVD.sav"
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
# Make predictions on the test set
predictions = loaded_model.test(testset)

# Calculate RMSE to evaluate model performance
rmse = accuracy.rmse(predictions)
print(f"RMSE of the SVD model: {rmse:.4f}")

RMSE: 0.7865
RMSE of the SVD model: 0.7865


In [8]:
from surprise import SVDpp, SlopeOne

# Instantiate the SVD++ model
svdpp = SVDpp()

# Train the model on the training set
svdpp.fit(trainset)

# Make predictions on the test set
predictions_svdpp = svdpp.test(testset)

# Calculate RMSE for SVD++
rmse_svdpp = accuracy.rmse(predictions_svdpp)
print(f"RMSE of the SVD++ model: {rmse_svdpp:.4f}")

# Instantiate the Slope One model
slope_one = SlopeOne()

# Train the model on the training set
slope_one.fit(trainset)

# Make predictions on the test set
predictions_slope_one = slope_one.test(testset)

# Calculate RMSE for Slope One
rmse_slope_one = accuracy.rmse(predictions_slope_one)
print(f"RMSE of the Slope One model: {rmse_slope_one:.4f}")


RMSE: 0.8134
RMSE of the SVD++ model: 0.8134
RMSE: 0.8551
RMSE of the Slope One model: 0.8551


In [9]:
# Save Model
filename = "../models/20M_Model_SVDPlusPlus.sav"
pickle.dump(svdpp, open(filename, "wb"))
filename = "../models/20M_Model_SlopeOne.sav"
pickle.dump(slope_one, open(filename, "wb"))

In [10]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate

from surprise.model_selection import GridSearchCV

# Define the parameter grid for SVD
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 50],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.1]
}

# Perform Grid Search Cross Validation
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)


# Fit the grid searchz6t
gs.fit(data)

# Get the best score and best parameters
print(f"Best RMSE score obtained: {gs.best_score['rmse']:.4f}")
print(f"Best parameters: {gs.best_params['rmse']}")


c:\Users\birne\anaconda3\envs\SEP24-BDS-INT-FILM-RECO\lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best RMSE score obtained: 0.7956
Best parameters: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [30]:
SVDgs = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)
SVDgs.fit(trainset)

In [32]:
# Save Model
filename = "../models/20M_Model_SVDGridCV.sav"
pickle.dump(SVDgs, open(filename, "wb"))

In [33]:
from surprise import KNNWithMeans, KNNBaseline

# Define item-based similarity options
sim_options = {
    'name': 'cosine',  # Can also try 'pearson'
    'user_based': False  # Use item-based collaborative filtering
}

# Instantiate and train the KNNWithMeans model
knn_means = KNNWithMeans(sim_options=sim_options)
knn_means.fit(trainset)

# Make predictions on the test set for KNNWithMeans
predictions_knn_means = knn_means.test(testset)

# Calculate RMSE for KNNWithMeans
rmse_knn_means = accuracy.rmse(predictions_knn_means)
print(f"RMSE of the KNNWithMeans model: {rmse_knn_means:.4f}")

# Instantiate and train the KNNBaseline model
knn_baseline = KNNBaseline(sim_options=sim_options)
knn_baseline.fit(trainset)

# Make predictions on the test set for KNNBaseline
predictions_knn_baseline = knn_baseline.test(testset)

# Calculate RMSE for KNNBaseline
rmse_knn_baseline = accuracy.rmse(predictions_knn_baseline)
print(f"RMSE of the KNNBaseline model: {rmse_knn_baseline:.4f}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8342
RMSE of the KNNWithMeans model: 0.8342
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8334
RMSE of the KNNBaseline model: 0.8334


In [34]:
# Save Model
filename = "../models/20M_Model_KNNWithMeans.sav"
pickle.dump(knn_means, open(filename, "wb"))
filename = "../models/20M_Model_KNNBaseline.sav"
pickle.dump(knn_baseline, open(filename, "wb"))


In [ ]:
from surprise.model_selection import RandomizedSearchCV
from surprise import SVD

# Define a refined parameter grid
param_grid = {
    'n_factors': [100, 125, 150, 175, 200],     # Number of latent factors
    'n_epochs': [30, 50, 70],                   # Number of epochs
    'lr_all': [0.005, 0.0075, 0.01],            # Learning rate for all parameters
    'reg_all': [0.05, 0.1, 0.15]                # Regularization term for all parameters
}

# Perform Randomized Search Cross Validation
rs = RandomizedSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_iter=10, random_state=42, n_jobs=-1)

# Fit the randomized search
rs.fit(data)
          
# Get the best score and best parameters
print(f"Best RMSE score obtained: {rs.best_score['rmse']:.4f}")
print(f"Best parameters: {rs.best_params['rmse']}")


c:\Users\birne\anaconda3\envs\SEP24-BDS-INT-FILM-RECO\lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [5]:
# Import the necessary module
from surprise import SVD

# Instantiate the SVD model with the best parameters from RandomizedSearchCV
best_svd = SVD(n_factors=200, n_epochs=70, lr_all=0.0075, reg_all=0.1)

# Train the model on the training set
best_svd.fit(trainset)

# Make predictions on the test set
predictions_best_svd = best_svd.test(testset)

# Calculate RMSE to evaluate the model performance
from surprise import accuracy
rmse_best_svd = accuracy.rmse(predictions_best_svd)
print(f"RMSE of the tuned SVD model: {rmse_best_svd:.4f}")


RMSE: 0.8169
RMSE of the tuned SVD model: 0.8169


In [6]:
# Save Model
filename = "../models/20M_Model_optimizedSVD.sav"
pickle.dump(best_svd, open(filename, "wb"))


In [8]:
filename = "../models/20M_Model_optimizedSVD.sav"
# load the model from disk
best_svd = pickle.load(open(filename, 'rb'))

#Step 2: Building a Hybrid Recommendation System

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Extract genres from the movies dataset
movies['genres'] = movies['genres'].str.replace('|', ' ')

# Use TF-IDF Vectorizer to convert genres into feature vectors
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

# Compute cosine similarity between movies based on genres
genres_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Display a sample similarity score matrix (first 5 movies)
print("Genres Similarity Matrix (first 5 movies):")
print(genres_similarity[:5, :5])


Genres Similarity Matrix (first 5 movies):
[[1.         0.81339357 0.15918745 0.14385421 0.26417526]
 [0.81339357 1.         0.         0.         0.        ]
 [0.15918745 0.         1.         0.90367808 0.60258272]
 [0.14385421 0.         0.90367808 1.         0.5445408 ]
 [0.26417526 0.         0.60258272 0.5445408  1.        ]]


# 2.1.1: Incorporate Tags into Content-Based Features

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Load the stop words
stop_words = set(stopwords.words('english'))

# Preprocess tags: Combine all tags for each movie
tags['tag'] = tags['tag'].fillna('').str.lower()
tags['tag'] = tags['tag'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))  # Remove non-alphabet characters
tags_grouped = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

# Merge the tags and movies datasets to combine genres and tags
movies_tags = pd.merge(movies, tags_grouped, on='movieId', how='left')
movies_tags['tag'] = movies_tags['tag'].fillna('')  # Replace NaN with an empty string

# Combine genres and tags into a single feature
movies_tags['combined_features'] = movies_tags['genres'] + ' ' + movies_tags['tag']

# Apply text preprocessing: Remove stopwords from the combined features
movies_tags['combined_features'] = movies_tags['combined_features'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words])
)

# Display a sample of the combined features
print("Combined Features for Movies (first 5 movies):")
print(movies_tags[['movieId', 'title', 'combined_features']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\birne\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Combined Features for Movies (first 5 movies):
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                   combined_features  
0  Adventure Animation Children Comedy Fantasy wa...  
1  Adventure Children Fantasy time travel adapted...  
2  Comedy Romance old people actually funny seque...  
3  Comedy Drama Romance chick flick revenge chara...  
4  Comedy diane keaton family sequel steve martin...  


#Step 2.1.2: Vectorize Combined Features and Calculate Similarity

In [6]:
# Use TF-IDF Vectorizer to convert the combined features into feature vectors
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_tags['combined_features'])

# Compute cosine similarity between movies based on combined features
combined_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Display a sample of the similarity matrix (first 5 movies)
print("Combined Similarity Matrix (first 5 movies):")
print(combined_similarity[:5, :5])


Combined Similarity Matrix (first 5 movies):
[[1.         0.06603966 0.01117023 0.0048979  0.03516531]
 [0.06603966 1.         0.00105999 0.06323018 0.02403781]
 [0.01117023 0.00105999 1.         0.02789314 0.11785211]
 [0.0048979  0.06323018 0.02789314 1.         0.0245282 ]
 [0.03516531 0.02403781 0.11785211 0.0245282  1.        ]]


#Integrate Content-Based and Collaborative Filtering for Hybrid Recommendations

#2.3: Calculate Hybrid Scores

In [9]:
def hybrid_recommendation(user_id, movie_id, combined_similarity, svd_model, movies_df, weight_cf=0.7, weight_cb=0.3):
    # Collaborative Filtering Score
    cf_score = svd_model.predict(user_id, movie_id).est

    # Content-Based Similarity Score
    # Find the index of the movie_id in movies dataframe
    try:
        movie_idx = movies_df[movies_df['movieId'] == movie_id].index[0]
    except IndexError:
        # If movie_id is not found in the dataset, return a score of 0
        return 0

    # Calculate the average similarity score for the given movie to all movies rated by the user
    user_rated_movies = ratings[ratings['userId'] == user_id]['movieId'].values
    if len(user_rated_movies) > 0:
        cb_score = np.mean([combined_similarity[movie_idx, movies_df[movies_df['movieId'] == m_id].index[0]] for m_id in user_rated_movies])
    else:
        cb_score = 0  # If no ratings by user, content-based score is 0

    # Hybrid Score: Weighted combination of CF and CB
    hybrid_score = (weight_cf * cf_score) + (weight_cb * cb_score)

    return hybrid_score

# Example usage: Predict a score for user 1 and movie 2
user_id = 1
movie_id = 2
hybrid_score = hybrid_recommendation(user_id, movie_id, combined_similarity, best_svd, movies_tags)
print(f"Hybrid recommendation score for user {user_id} and movie {movie_id}: {hybrid_score:.4f}")


Hybrid recommendation score for user 1 and movie 2: 2.5895


#Step 2.4: Generate Recommendations for a User

In [10]:
def generate_hybrid_recommendations(user_id, combined_similarity, svd_model, movies_df, ratings_df, num_recommendations=10, weight_cf=0.7, weight_cb=0.3):
    # Get the list of all movie IDs
    all_movie_ids = movies_df['movieId'].values

    # Get the list of movies rated by the user
    rated_movie_ids = ratings_df[ratings_df['userId'] == user_id]['movieId'].values

    # Filter out movies that the user has already rated
    movies_to_recommend = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movie_ids]

    # Calculate hybrid scores for all movies the user has not rated
    hybrid_scores = []
    for movie_id in movies_to_recommend:
        score = hybrid_recommendation(user_id, movie_id, combined_similarity, svd_model, movies_df, weight_cf, weight_cb)
        hybrid_scores.append((movie_id, score))

    # Sort movies by hybrid score in descending order
    hybrid_scores.sort(key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    top_movies = hybrid_scores[:num_recommendations]

    # Display the top recommendations with movie titles
    recommendations = []
    for movie_id, score in top_movies:
        movie_title = movies_df[movies_df['movieId'] == movie_id]['title'].values[0]
        recommendations.append((movie_title, score))

    return recommendations

# Example usage: Generate top 10 recommendations for user 1
user_id = 1
num_recommendations = 10
top_recommendations = generate_hybrid_recommendations(user_id, combined_similarity, best_svd, movies_tags, ratings, num_recommendations)

# Display the recommendations
print(f"Top {num_recommendations} movie recommendations for user {user_id}:")
for idx, (title, score) in enumerate(top_recommendations, start=1):
    print(f"{idx}. {title} (Score: {score:.4f})")


Top 10 movie recommendations for user 1:
1. Octopus, The (Le poulpe) (1998) (Score: 3.3471)
2. Marihuana (1936) (Score: 3.3176)
3. Zero Motivation (Efes beyahasei enosh) (2014) (Score: 3.3119)
4. Playing the Victim (Izobrazhaya zhertvu) (2006) (Score: 3.2837)
5. Welfare (1975) (Score: 3.2223)
6. Car Bonus (Autobonus) (2001) (Score: 3.2114)
7. Johnny Express (2014) (Score: 3.2109)
8. I Belong (Som du ser meg) (2012) (Score: 3.2019)
9. Shepard & Dark (2012) (Score: 3.1959)
10. Last Days in Vietnam (2014) (Score: 3.1821)


#Evaluate Hybrid Model Performance

In [11]:
def precision_recall_at_k(user_id, combined_similarity, svd_model, movies_df, ratings_df, k=10, weight_cf=0.7, weight_cb=0.3):
    # Generate top K hybrid recommendations for the user
    top_recommendations = generate_hybrid_recommendations(user_id, combined_similarity, svd_model, movies_df, ratings_df, num_recommendations=k, weight_cf=weight_cf, weight_cb=weight_cb)
    recommended_movie_ids = [movies_df[movies_df['title'] == title]['movieId'].values[0] for title, score in top_recommendations]

    # Get the list of movies rated by the user
    user_ratings = ratings_df[ratings_df['userId'] == user_id]

    # Define a relevant rating threshold (e.g., ratings >= 4 are considered relevant)
    relevant_ratings = user_ratings[user_ratings['rating'] >= 4.0]
    relevant_movie_ids = relevant_ratings['movieId'].values

    # Calculate Precision and Recall
    relevant_recommended = [movie_id for movie_id in recommended_movie_ids if movie_id in relevant_movie_ids]
    precision = len(relevant_recommended) / len(recommended_movie_ids) if len(recommended_movie_ids) > 0 else 0
    recall = len(relevant_recommended) / len(relevant_movie_ids) if len(relevant_movie_ids) > 0 else 0

    return precision, recall

# Example usage: Calculate precision and recall for user 1 for top 10 recommendations
user_id = 1
precision, recall = precision_recall_at_k(user_id, combined_similarity, best_svd, movies_tags, ratings, k=10)
print(f"Precision@10 for user {user_id}: {precision:.4f}")
print(f"Recall@10 for user {user_id}: {recall:.4f}")


Precision@10 for user 1: 0.0000
Recall@10 for user 1: 0.0000


#Evaluating Precision and Recall for Multiple Users

In [ ]:
# def evaluate_precision_recall(users, combined_similarity, svd_model, movies_df, ratings_df, k=10, weight_cf=0.7, weight_cb=0.3):
#     total_precision = 0
#     total_recall = 0
#     count_users = 0

#     for user_id in users:
#         # Calculate precision and recall for the given user
#         precision, recall = precision_recall_at_k(user_id, combined_similarity, svd_model, movies_df, ratings_df, k, weight_cf, weight_cb)

#         # Accumulate precision and recall
#         total_precision += precision
#         total_recall += recall
#         count_users += 1

#     # Calculate average precision and recall
#     avg_precision = total_precision / count_users if count_users > 0 else 0
#     avg_recall = total_recall / count_users if count_users > 0 else 0

#     return avg_precision, avg_recall

# # Example usage: Evaluate precision and recall for the first 100 users
# user_ids = ratings['userId'].unique()[:100]  # Get the first 100 unique user IDs
# avg_precision, avg_recall = evaluate_precision_recall(user_ids, combined_similarity, best_svd, movies_tags, ratings, k=10)

# print(f"Average Precision@10 for the first 100 users: {avg_precision:.4f}")
# print(f"Average Recall@10 for the first 100 users: {avg_recall:.4f}")


#Finding the Best Weights for Hybrid Recommendation

In [12]:
def find_best_weights(users, combined_similarity, svd_model, movies_df, ratings_df, k=10):
    weight_combinations = [
        (0.1, 0.9), (0.2, 0.8), (0.3, 0.7), (0.4, 0.6), (0.5, 0.5),
        (0.6, 0.4), (0.7, 0.3), (0.8, 0.2), (0.9, 0.1)
    ]
    best_precision = 0
    best_recall = 0
    best_weights = None

    # Evaluate each combination of weights
    for weight_cf, weight_cb in weight_combinations:
        avg_precision, avg_recall = evaluate_precision_recall(
            users, combined_similarity, svd_model, movies_df, ratings_df, k, weight_cf, weight_cb
        )

        print(f"Weights CF: {weight_cf}, CB: {weight_cb} -> Average Precision@{k}: {avg_precision:.4f}, Average Recall@{k}: {avg_recall:.4f}")

        # Find the best combination based on precision and recall
        if avg_precision > best_precision or (avg_precision == best_precision and avg_recall > best_recall):
            best_precision = avg_precision
            best_recall = avg_recall
            best_weights = (weight_cf, weight_cb)

    return best_weights, best_precision, best_recall

# Example usage: Find the best weights for the first 50 users
user_ids = ratings['userId'].unique()[:50]  # Get the first 50 unique user IDs
best_weights, best_precision, best_recall = find_best_weights(user_ids, combined_similarity, best_svd, movies_tags, ratings, k=10)

# Display the best weights and their corresponding performance metrics
print(f"\nBest weights (CF, CB): {best_weights}")
print(f"Best Average Precision@10: {best_precision:.4f}")
print(f"Best Average Recall@10: {best_recall:.4f}")


NameError: name 'evaluate_precision_recall' is not defined